# TP 2 de Statistique descriptive avec SAS

### Les formats

Un format est un *masque d'affichage des données*. Cet objet permet en SAS de faciliter la lecture des tables. 

Il existe deux types de format : les formats (aussi appelés les formats de lecture) et les informats (ou format d'écriture).

L'exemple typique d'utilisation de format est celui des formats de date. Par défaut, SAS stocke les dates sous la forme d'un entier qui correspond au nombre de jours entre la date et le 1er janvier 1960. Il s'agit donc d'une variable numérique. Il est très délicat pour un utilisateur de manipuler ces variables sous cette forme (à quoi correspond par exemple la date -4017?). Les formats de date permettent de changer l'affichage de la variable : on peut par exemple choisir d'afficher la date sous la forme Jour/Mois/Année à l'aide du format DDMMYY. ou sous la forme MoisAnnée à l'aide du format MONYY.

Les formats ne s'appliquent pas uniquement sur les dates mais sur n'importe quelle variable. On peut par exemple vouloir afficher des numéros de départements sur trois positions en complétant par des zéros à gauche (001, 002 ...) ou afficher un nombre restreint de décimales.

Une liste des formats est disponible ici.

Pour appliquer un format à une variable, on utilisera l'instruction FORMAT dans une procédure ou dans une étape DATA.

Par exemple, prenons la table AIR de la bibliothèque SASHELP.

Il est possible de créer 

## Exercice 1 : Analyse de l'âge et de la catégorie socioprofessionnelle des habitants d'Ille-et-Vilaine

**Question 1** : Créez une bibliothèque *source* en lecture seule et une bibliothèque *resultat* ayant vocation à contenir les résultats de vos analyses.

Je pense que certains étudiants ont encore des difficultés avec ça...

In [1]:
LIBNAME source "/folders/myfolders/Donnees_TP_SAS/TP2/Source" ACCESS = READONLY;
LIBNAME resultat "/folders/myfolders/Donnees_TP_SAS/TP2/Resultat" ;

SAS Connection established. Subprocess id is 2235



In [ ]:
PROC PRINT DATA = source.rp12_d35(obs = 20);
RUN;

Je pense supprimer cetet question **Question 1** : À l'aide d'une étape DATA, ajoutez la variable AGETRANCHE qui va valoir et stockez le résultat dans la bibliothèque *resultat* : 
* "[00;18[" quand l'âge est entre 0 et 18 ans exclus
* "[18;26[" quand l'âge est entre 18 et 26 ans
* "[26;30[" quand l'âge est entre 26 et 30 ans
* "[30;60[" quand l'âge est entre 30 et 60 ans
* "[60;65[" quand l'âge est entre 60 et 65 ans
* "[65;80[" quand l'âge est entre 65 et 80 ans
* "[80 et plus[" sinon


In [ ]:
DATA resultat.RP12_d35;
SET source.RP12_d35;
SELECT;
    WHEN (AGE >= 0 AND AGE < 18) AGETRANCHE = "[00;18]";
    WHEN (AGE <= 18 AND AGE < 26) AGETRANCHE = "[18;26[";
    WHEN (AGE <= 26 AND AGE < 40) AGETRANCHE = "[26;40[";
    WHEN (AGE <= 40 AND AGE < 55) AGETRANCHE = "[40;55[";
    WHEN (AGE <= 55 AND AGE < 65) AGETRANCHE = "[55;65[";
    WHEN (AGE <= 65 AND AGE < 80) AGETRANCHE = "[65;80[";
    OTHERWISE AGETRANCHE = "[80;+[";
END;
RUN;

In [ ]:
PROC PRINT DATA = resultat.rp12_d35(obs = 20);
RUN;

**Question 2** : Créez un format NOMTRANCHE qui sera appliqué à la variable AGE tel qu'on associe :
* Si $\text{AGE} \in [00;18[" \longrightarrow$ "Enfants"
* Si $\text{AGE} \in [18;26[" \longrightarrow$ "Etudiants"
* Si $\text{AGE} \in [26;40[" \longrightarrow$ "Jeunes actifs"
* Si $\text{AGE} \in [40;55[" \longrightarrow$ "Actifs"
* Si $\text{AGE} \in [55;65[" \longrightarrow$ "Actifs expérimentés"
* Si $\text{AGE} \in [65;80[" \longrightarrow$ "Jeunes seniors"
* Si $\text{AGE} \in [80;+\infty[" \longrightarrow$ "Seniors"

Où est stocké le format NOMTRANCHE ?

In [11]:
PROC FORMAT;
    VALUE NOMTRANCHE
        LOW - 17 = "Enfants"
        18 - 25 = "Etudiants"
        26 - 39 = "Jeunes actifs"
        40 - 54 = "Actifs"
        55 - 64 = "Actifs expérimentés"
        65 - 79 = "Jeunes seniors"
        80 - HIGH = "Seniors";
RUN;

**Question 3** : Appliquez le format NOMTRANCHE à la variable AGETRANCHE.

In [23]:
OPTIONS FMTSEARCH=(work);

DATA resultat.rp12_d35;
SET resultat.rp12_d35;
FORMAT AGETRANCHE NOMTRANCHE.;
RUN;

In [24]:
PROC PRINT DATA = resultat.rp12_d35(obs = 30);
RUN;

Obs,IPONDI,CS1,SEXE,TACT,AGE,ECM,cumul,AGETRANCHE
1,4.08412,4,1,12,35,104.426,104.43,[40;55
2,4.08412,5,2,11,34,100.342,204.77,[40;55
3,4.08412,8,2,23,2,-30.350,174.42,[0;18]
4,4.25597,7,2,21,72,267.911,442.33,[80;+[
5,4.16916,7,1,21,79,290.845,733.17,[80;+[
6,4.16916,8,2,24,80,295.014,1028.19,[80;+[
7,4.00082,5,1,11,56,185.528,1213.72,[65;80
8,4.00082,7,2,21,53,173.525,1387.24,[55;65
9,4.00082,4,1,11,22,49.500,1436.74,[26;40
10,4.00082,8,1,22,19,37.497,1474.24,[26;40


**Question 4** : Importez le format NOMTACT.

**Question 5** : Appliquez le format NOMTACT à la variable TACT.

**Question 6** : Réalisez le tableau de contingence des variables AGETRANCHE et TACT.

In [ ]:
**Question 

## Exercice 2 : 

Dans cet exercice, on considère la table *pauv12dep.sas7bdat*.